### analysis the telemetry and timing info

build detection rules for pitstop and crash events


# Imports

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="7"


## Load Data

In [2]:
import os
os.getcwd()


'/scratch/hpda/indycar/predictor/notebook/5.DeployModel'

indy2018-pitstops is the dataset of \$C command with two more columns

1. timestamp, the latest timestamp before this $C command is recieved
2. lapdist, the latest lap_distance record from the telemetry of this particular car


In [3]:
# this file ware created from the log
# <latest_timestamp> <latest_lapdistance> $C 
pitstops = pd.read_csv('indy2018-pitstops.csv')
data = pitstops.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
pitstops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398 entries, 0 to 1397
Data columns (total 23 columns):
timestamp               1398 non-null object
lapdist                 1398 non-null float64
rank                    1398 non-null int64
car_number              1398 non-null int64
unique_id               1398 non-null object
completed_laps          1398 non-null int64
elapsed_time            1398 non-null float64
last_laptime            1398 non-null float64
lap_status              1398 non-null object
best_laptime            1398 non-null float64
best_lap                1398 non-null object
time_behind_leader      1398 non-null float64
laps_behind_leade       1398 non-null object
time_behind_prec        1398 non-null float64
laps_behind_prec        1398 non-null object
overall_rank            1398 non-null object
overall_best_laptime    1398 non-null float64
current_status          1398 non-null object
track_status            1398 non-null object
pit_stop_count          1398 non-n

In [4]:
# to find 'strage' pitstops when they pit not near the pitlane entry
#data[(data['lapdist'] > 500) and (data['lapdist'] < 4000)]
data[(data['lapdist'] > 300)][(data['lapdist'] < 4000)]

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,timestamp,lapdist,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
188,18:22:35.885,2964.75,26,17,20,140,7170.2231,120.3700,P,41.0610,...,12.0121,0,1A,41.0610,Active,Y,6,8C,21,0
191,18:26:20.326,2964.75,26,17,20,142,7394.4007,123.6573,P,41.0610,...,28.3470,0,1A,41.0610,Active,Y,7,8E,21,0
449,19:16:53.390,3818.00,22,21,5,191,10427.4875,120.6266,P,40.8170,...,2.9782,0,16,40.8170,Active,Y,8,BF,6,3
608,16:45:42.847,330.16,32,26,18,31,1356.9691,69.0226,P,41.9395,...,1.8259,0,20,41.9395,Active,G,1,1F,19,0
611,17:06:56.916,334.03,28,26,18,53,2631.0479,127.2128,P,41.0509,...,0.3681,0,1C,41.0509,Active,Y,2,35,19,0
617,17:08:42.914,332.73,30,26,18,54,2736.8594,105.8115,P,41.0509,...,8.6727,0,1E,41.0509,Active,Y,3,36,19,0
620,17:26:57.902,333.00,27,26,18,70,3831.9342,123.3099,P,41.0509,...,3.4335,0,1B,41.0509,Active,Y,4,46,19,0
623,17:51:54.808,330.75,24,26,18,101,5328.9025,73.1547,P,41.0509,...,3.7780,0,18,41.0509,Active,G,5,65,19,0
628,18:15:43.236,332.06,26,26,18,134,6757.2836,79.3177,P,40.8811,...,3.1213,0,1A,40.8811,Active,G,6,86,19,0
632,18:37:27.516,331.44,25,26,18,150,8061.7088,105.8029,P,40.8811,...,18.0769,0,19,40.8811,Active,Y,7,96,19,0


### Anomaly of pitstops

1. car 17. pitstop entry is weired at '2964.75', the root cause is that the telemetry lost from 18:21:57.732 to 18:26:32.928

2. car 26. pitstio entry is as large as '330.16' at lap 31, not near the pit lane entry. By investigating the telemetry, it seems that the lap_distance value for this car is not well calibrated. The maximum value of lap_distance of previous lap, lap 30, is only '3734' 


In [5]:

pitstops[pitstops['car_number']==17]

,timestamp,lapdist,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
166,16:47:58.640,249.34,28,17,20,34,1492.8432,76.4384,P,42.0016,...,11.1614,0,1C,42.0016,Active,G,1,22,21,0
167,16:48:29.828,1905.80,28,17,20,34,1492.8432,76.4384,P,42.0016,...,11.2787,0,1C,42.0016,Active,G,1,22,21,0
168,16:48:36.075,2467.78,28,17,20,34,1492.8432,76.4384,P,42.0016,...,11.2787,0,1C,42.0016,Active,G,1,22,21,0
169,16:48:44.052,3238.73,28,17,20,34,1492.8432,76.4384,P,42.0016,...,0.0000,1,1C,42.0016,Active,G,1,22,21,0
170,16:48:46.831,3518.17,31,17,20,34,1492.8432,76.4384,P,42.0016,...,0.0000,1,1F,42.0016,Active,G,1,22,21,0
171,16:48:50.806,3888.69,32,17,20,34,1492.8432,76.4384,P,42.0016,...,0.0000,1,20,42.0016,Active,G,1,22,21,0
172,17:07:02.303,255.20,30,17,20,53,2636.1781,127.4500,P,41.0610,...,4.6566,0,1E,41.0610,Active,Y,2,35,21,0
173,17:08:06.842,3175.95,30,17,20,53,2636.1781,127.4500,P,41.0610,...,4.6566,0,1E,41.0610,Active,Y,2,35,21,0
174,17:08:31.731,4214.02,30,17,20,53,2636.1781,127.4500,P,41.0610,...,5.1302,0,1E,41.0610,Active,Y,2,35,21,0
175,17:26:58.632,256.34,28,17,20,70,3832.4968,125.1808,P,41.0610,...,0.5626,0,1C,41.0610,Active,Y,3,46,21,0
